## Merge Two files into Single File
#### Check the name of common column and edit it if you need

In [ ]:
import pandas as pd

file_a_path = 'BreastCancer_addDummyColumn.csv'
file_b_path = 'BreastCancer_addCountColumns_Primary_Secondary.csv'

df_a = pd.read_csv(file_a_path)
df_b = pd.read_csv(file_b_path)

merged_df = pd.merge(df_a, df_b, left_on='protocolSection.identificationModule.nctId', right_on='NCT number', how='inner')

merged_df.to_csv('BreastCancer_Merged_addDummyColumn_addCountColumns_Primary_Secondary.csv', index=False)

#### If you execute above cell, you can see the same data with different column in the single file. 
#### Drop one of those column to be looked clean.

In [ ]:
merged_df.drop(columns=['NCT number'], inplace=True)
merged_df.to_csv('BreastCancer_Merged_addDummyColumn_addCountColumns_Primary_Secondary.csv', index=False)

## When working on large files multiple times, it's more efficient to preload the file before starting your tasks, as this can save a lot of time.
## Adjust the conditions as needed based on the situation.

In [ ]:
import pandas as pd
import ast

df = pd.read_csv('merged_Dump.csv')  # Replace with your actual file path

# Adjust the conditions. Belows are examples
df = df[df['protocolSection.designModule.designInfo.interventionModel'] == 'PARALLEL']
df = df[(df['breast_cancer_dummy'] == 1)]

#### Filter the data if the interventionmodel is parallel and condtions are breast cancer.
#### Scope Down the Primary data
#### Normalize the measure title using function and calculate the frequency of measure title.

In [ ]:
import pandas as pd
import ast
import re
from collections import Counter

# Load the CSV file
df = pd.read_csv('your_data_file.csv')  # Replace with your actual file path

# Filter rows where intervention model is 'PARALLEL'
df = df[df['protocolSection.designModule.designInfo.interventionModel'] == 'PARALLEL']

# Function to convert string representation of lists and dicts to Python objects
def process_data(data):
    if pd.isna(data):
        return []
    try:
        return ast.literal_eval(data)
    except (ValueError, SyntaxError):
        return []

# Function to check for 'breast cancer' presence
def has_breast_cancer(data):
    return 'breast cancer' in data.lower()

# Categorize the measure title for relevance
def categorize_string(s):
    patterns = {
        'PFS': r'progression[-\s]*free[-\s]*survival|pfs',
        'DFS': r'disease[-\s]*free[-\s]*survival|dfs',
        'OS': r'\boverall[-\s]*survival\b|\bos\b',
        'ORR': r'objective[-\s]*response[-\s]*rate|overall[-\s]*response[-\s]*rate|orr\b'
    }
    s = s.lower()
    for key, pattern in patterns.items():
        if re.search(pattern, s):
            return key
    return 'Others'  # Return 'Others' if no category matches

# Processing function for outcome measures
def process_outcome_measures(data):
    outcomes = process_data(data)
    titles = [outcome.get('title', '') for outcome in outcomes if outcome.get('type') == 'PRIMARY']
    return [categorize_string(title) for title in titles]

# Filter rows containing 'breast cancer' and process outcome measures
df['hasBreastCancer'] = df.apply(
    lambda row: has_breast_cancer(
        process_conditions(row['protocolSection.conditionsModule.conditions']),
        process_conditions(row['derivedSection.conditionBrowseModule.meshes'])
    ), axis=1)

filtered_df = df[df['hasBreastCancer']]

# Process outcome measures for filtered rows
filtered_df['Processed Titles'] = filtered_df['resultsSection.outcomeMeasuresModule.outcomeMeasures'].apply(process_outcome_measures)

# Flatten the list of titles and count the occurrences
title_counter = Counter([title for sublist in filtered_df['Processed Titles'] for title in sublist])

# Create a DataFrame from the counter for easy handling
title_stats_df = pd.DataFrame(title_counter.items(), columns=['Title', 'Count'])
title_stats_df['Frequency'] = (title_stats_df['Count'] / title_stats_df['Count'].sum() * 100).round(3)

# Optionally, save to CSV
title_stats_df.to_csv('breast_cancer_titles_frequency.csv', index=False)

print(title_stats_df)


In [ ]:
# Function to check for 'cancer' in a list of strings
def has_cancer_in_list(conditions_list):
    # Check if the input is NaN or not a string before processing
    if pd.isna(conditions_list) or not isinstance(conditions_list, str):
        return False
    try:
        conditions_list = ast.literal_eval(conditions_list)
    except (ValueError, SyntaxError):
        return False
    return any('cancer' in condition.lower() for condition in conditions_list if isinstance(conditions_list, list))

def has_cancer_in_dict_list(meshes_list):
    # Check if the input is NaN or not a string before processing
    if pd.isna(meshes_list) or not isinstance(meshes_list, str):
        return False
    try:
        meshes_list = ast.literal_eval(meshes_list)
    except (ValueError, SyntaxError):
        return False
    return any('cancer' in mesh['term'].lower() for mesh in meshes_list if isinstance(meshes_list, list) and isinstance(mesh, dict) and 'term' in mesh)

# Apply the functions to filter the DataFrame where 'hasResult' is True or False
df['hasCancer'] = df.apply(lambda row: has_cancer_in_list(row['protocolSection.conditionsModule.conditions']) or
                                          has_cancer_in_dict_list(row['derivedSection.conditionBrowseModule.meshes']), axis=1)

# Count occurrences of 'hasCancer' when 'hasResult' is True
count_true = df[df['hasResults'] == True]['hasCancer'].sum()
# Count occurrences of 'hasCancer' when 'hasResult' is False
count_false = df[df['hasResults'] == 'FALSE']['hasCancer'].sum()

# Print the counts
print("Number of 'hasCancer' when 'hasResult' is True:", count_true)
print("Number of 'hasCancer' when 'hasResult' is False:", count_false)

In [ ]:
import pandas as pd
import ast

# Define function to process conditions and convert them from string to list if needed
def process_conditions(data):
    if pd.isna(data):
        return []
    if isinstance(data, str):
        try:
            data = ast.literal_eval(data)
        except (ValueError, SyntaxError):
            return []
    return data

# Define function to check for 'breast cancer' in a list of strings or dictionaries
def has_breast_cancer(conditions_list, dict_list):
    if any('breast cancer' in condition.lower() for condition in conditions_list):
        return True
    if any('breast cancer' in mesh['term'].lower() for mesh in dict_list if 'term' in mesh):
        return True
    return False

# Load the CSV file
# df = pd.read_csv('Cancer_Dump_noResult.csv')

# Apply the functions to check for breast cancer presence in each row
df['hasBreastCancer'] = df.apply(
    lambda row: has_breast_cancer(
        process_conditions(row['protocolSection.conditionsModule.conditions']),
        process_conditions(row['derivedSection.conditionBrowseModule.meshes'])
    ), axis=1)

# Count occurrences of 'hasBreastCancer' when 'hasResults' is True
count_true = df[df['hasResults'] == True]['hasBreastCancer'].sum()
# Count occurrences of 'hasBreastCancer' when 'hasResults' is False
count_false = df[df['hasResults'] == False]['hasBreastCancer'].sum()

# Print the results
print("Number of 'breast cancer' entries when 'hasResults' is True:", count_true)
print("Number of 'breast cancer' entries when 'hasResults' is False:", count_false)


In [ ]:
import pandas as pd
import ast
import re
from collections import Counter

# Function to convert string representation of lists and dicts to Python objects
def process_data(data):
    if pd.isna(data):
        return []
    try:
        return ast.literal_eval(data)
    except (ValueError, SyntaxError):
        return []

# Function to check for 'breast cancer' presence
def has_breast_cancer(data):
    return 'breast cancer' in data.lower()

# Categorize the measure title for relevance
def categorize_string(s):
    patterns = {
        'PFS': r'progression[-\s]*free[-\s]*survival|pfs',
        'DFS': r'disease[-\s]*free[-\s]*survival|dfs',
        'OS': r'\boverall[-\s]*survival\b|\bos\b',
        'ORR': r'objective[-\s]*response[-\s]*rate|overall[-\s]*response[-\s]*rate|orr\b'
    }
    s = s.lower()
    for key, pattern in patterns.items():
        if re.search(pattern, s):
            return key
    return 'others'

# Processing function for outcome measures
def process_outcome_measures(data):
    outcomes = process_data(data)
    # titles = [outcome.get('title', '') for outcome in outcomes if outcome.get('type') == 'PRIMARY']
    titles = [outcome.get('measure', '') for outcome in outcomes]
    return [categorize_string(title) for title in titles if categorize_string(title)]

# # Filter rows containing 'breast cancer' and process outcome measures
# df['hasBreastCancer'] = df.apply(lambda row: any(has_breast_cancer(cond) for cond in process_data(row['protocolSection.conditionsModule.conditions'])) or
#                                                   any(has_breast_cancer(mesh['term']) for mesh in process_data(row['derivedSection.conditionBrowseModule.meshes']) if 'term' in mesh), axis=1)

# filtered_df = df[df['hasBreastCancer']]

# Process outcome measures for filtered rows
df['Processed Titles'] = df['protocolSection.outcomesModule.secondaryOutcomes'].apply(process_outcome_measures)

# Flatten the list of titles and count the occurrences
title_counter = Counter([title for sublist in df['Processed Titles'] for title in sublist])

# Create a DataFrame from the counter for easy handling
title_stats_df = pd.DataFrame(title_counter.items(), columns=['Title', 'Count'])
title_stats_df['Frequency'] = (title_stats_df['Count'] / title_stats_df['Count'].sum() * 100).round(3)
print(title_stats_df)
# Optionally, save to CSV
# title_stats_df.to_csv('breast_cancer_noResult_primary_title_counts.csv', index=False)

In [ ]:
import pandas as pd
import re

df = pd.read_csv('Valid_Parallel_with_DrugId_and_Disease.csv')

# Include the Primary/Secondary but exclude the Difference in "Column name"
primary_columns = [col for col in df.columns if 'PRIMARY' in col and 'Differences' not in col]
secondary_columns = [col for col in df.columns if 'SECONDARY' in col and 'Differences' not in col]

def extract_number(col):
    match = re.search(r'(PRIMARY|SECONDARY)(\d+)', col)
    if match:
        return int(match.group(2))
    return None

def max_numbered_non_null(row, columns):
    max_index = 0
    for col in columns:
        if pd.notna(row[col]):
            current_index = extract_number(col)
            if current_index > max_index:
                max_index = current_index
    return max_index 
    
df['Max_Primary'] = df.apply(lambda row: max_numbered_non_null(row, primary_columns), axis=1)
df['Max_Secondary'] = df.apply(lambda row: max_numbered_non_null(row, secondary_columns), axis=1)

# Calculate the sum of Max_Primary_Number per Disease ID and Disease Name
disease_summary = df.groupby(['Disease ID', 'Disease Name'])['Max_Secondary'].sum().reset_index(name='Sum of Max_Primary_Number')

# Sort the results in descending order of Sum of Max_Primary_Number and extract the top 3 entries.
top_3_diseases = disease_summary.sort_values(by='Sum of Max_Primary_Number', ascending=False).head(3)

print("Top 3 Disease IDs with the highest sum of Max_Primary_Number:")
print(top_3_diseases)

In [ ]:
import pandas as pd
import re

df = pd.read_csv('Output_Valid_SingleGroup.csv')

primary_columns = [col for col in df.columns if 'PRIMARY' in col and 'Differences' not in col]
secondary_columns = [col for col in df.columns if 'SECONDARY' in col and 'Differences' not in col]

def extract_number(col):
    match = re.search(r'(PRIMARY|SECONDARY)(\d+)', col)
    if match:
        return int(match.group(2))
    return None

def max_numbered_non_null(row, columns):
    max_index = 0
    for col in columns:
        if pd.notna(row[col]):
            current_index = extract_number(col)
            if current_index > max_index:
                max_index = current_index
    return max_index 
    
df['Max_Primary'] = df.apply(lambda row: max_numbered_non_null(row, primary_columns), axis=1)
df['Max_Secondary'] = df.apply(lambda row: max_numbered_non_null(row, secondary_columns), axis=1)

primary_min = df['Max_Primary'].min()
primary_max = df['Max_Primary'].max()
primary_avg = df['Max_Primary'].mean()
primary_std = df['Max_Primary'].std()

secondary_min = df['Max_Secondary'].min()
secondary_max = df['Max_Secondary'].max()
secondary_avg = df['Max_Secondary'].mean()
secondary_std = df['Max_Secondary'].std()

print(f"Primary - Min: {primary_min}, Max: {primary_max}, Avg: {primary_avg:.2f}, Std : {primary_std:.2f}")
print(f"Secondary - Min: {secondary_min}, Max: {secondary_max}, Avg: {secondary_avg:.2f}, Std : {secondary_std:.2f}")

### Make a graph

In [ ]:
import matplotlib.pyplot as plt

# Plotting Max_Primary
plt.figure(figsize=(10, 6))
plt.hist(df['Max_Primary'], bins=range(100,301), color='green', alpha=0.7, edgecolor='black')
plt.title('Distribution of the number of Primary Outcomes in Valid SingleGroup')
plt.xlabel('Primary Outcomes')
plt.ylabel('Trials')
# plt.xticks(range(1, df['Max_Primary'].max() + 1))
plt.grid(True)
plt.savefig('PrimaryOutcomes_ValidSingelGroup_100_300.png')
print('Save the graph of PrimaryOutcome')
plt.show()

# # Plotting Max_Secondary
plt.figure(figsize=(10, 6))
plt.hist(df['Max_Secondary'], bins=range(1, df['Max_Secondary'].max() + 2), color='red', alpha=0.7, edgecolor='red')
plt.title('Distribution of the number of Secondary Outcomes')
plt.xlabel('Secondary Outcomes')
plt.ylabel('Trials')
# plt.xticks(range(1, df['Max_Secondary'].max() + 1))
plt.grid(True)
plt.savefig('SecondaryOutcomes_ValidParallel.png')
print('Save the graph of SecondaryOutcome')
plt.show()

In [ ]:
primary_zero_nct = df[df['Max_Primary'] == 0]['NCT number']
secondary_zero_nct = df[df['Max_Secondary'] == 0]['NCT number']
both_zero_nct = df[(df['Max_Primary'] == 0) & (df['Max_Secondary'] == 0)]['NCT number']

both_zero_nct.to_csv('Primary_Secondary_BothZero_NCT_Numbers_ValidParallel.txt', index=False, header=None)
print("Completed writing NCT numbers to text files.")

In [ ]:
# Calculate the distribution of Max_Primary
primary_distribution = df['Max_Primary'].value_counts().sort_index()

# Convert the Series to DataFrame for easier CSV output
primary_distribution_df = primary_distribution.reset_index()
primary_distribution_df.columns = ['# of Primary', '# of Trials']

# Save the distribution to a CSV file
primary_distribution_df.to_csv('Primary_Distribution_ValidSingleGroup.csv', index=False)
print("Max_Primary distribution has been saved to Max_Primary_Distribution.csv.")

# Calculate the distribution of Max_Primary
primary_distribution = df['Max_Secondary'].value_counts().sort_index()

# Convert the Series to DataFrame for easier CSV output
primary_distribution_df = primary_distribution.reset_index()
primary_distribution_df.columns = ['# of Secondary', '# of Trials']

# Save the distribution to a CSV file
primary_distribution_df.to_csv('Secondary_Distribution_ValidSingleGroup.csv', index=False)
print("Max_Secondary distribution has been saved to Max_Secondary_Distribution.csv.")

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import re

df = pd.read_csv('0501_ValidParallel_Sample10.csv')

primary_columns = [col for col in df.columns if 'PRIMARY' in col and 'Differences' not in col]
secondary_columns = [col for col in df.columns if 'SECONDARY' in col and 'Differences' not in col]

def extract_number(col):
    match = re.search(r'(PRIMARY|SECONDARY)(\d+)', col)
    if match:
        return int(match.group(2))
    return None

def max_numbered_non_null(row, columns):
    max_index = 0
    for col in columns:
        if pd.notna(row[col]):
            current_index = extract_number(col)
            if current_index > max_index:
                max_index = current_index
    return max_index 
    
df['Max_Primary'] = df.apply(lambda row: max_numbered_non_null(row, primary_columns), axis=1)
df['Max_Secondary'] = df.apply(lambda row: max_numbered_non_null(row, secondary_columns), axis=1)

print(df[['NCT number', 'Max_Primary', 'Max_Secondary']])
# #Graph
# plt.figure(figsize=(10, 6))
# plt.plot(primary_count_distribution.index, primary_count_distribution.values, color='skyblue')
# plt.xlabel('Primary Outcomes')
# plt.ylabel('Trials')
# plt.title('Number of Trials per primary Outcomes in Valid Parallel')
# plt.grid(True)
# plt.show()
# plt.savefig('PrimaryOutcome_ValidParallel.png')

# # Calculate descriptive statistics
# min_value = df['Primary_Count'].min()
# max_value = df['Primary_Count'].max()
# mean_value = df['Primary_Count'].mean()
# std_dev = df['Primary_Count'].std()

# # Print descriptive statistics
# print(f"Minimum number of non-null PRIMARY entries: {min_value}")
# print(f"Maximum number of non-null PRIMARY entries: {max_value}")
# print(f"Mean number of non-null PRIMARY entries: {mean_value:.2f}")
# print(f"Standard deviation: {std_dev:.2f}")
# primary_zero_nct = df[df['Max_Primary_Number'] == 0]['NCT number']
# primary_zero_nct = df[df['Primary_Count']==1026]['NCT number']
# print(primary_zero_nct)

In [ ]:
import pandas as pd

# read .csv file
df = pd.read_csv('Output_Valid_SingleGroup.csv')

# calculate the mean of columne 'seriousEvents'
serious_events_mean = df['serious Adverse Events'].mean()
serious_events_max = df['serious Adverse Events'].max()
serious_events_min = df['serious Adverse Events'].min()
serious_events_std = df['serious Adverse Events'].std()

print("Mean of seriousEvents :", serious_events_mean)
print("Max of seriousEvents:", serious_events_max)
print("Min of seriousEvents:", serious_events_min)
print("Std of seriousEvents:", serious_events_std)

phase_counts = df['Phase'].value_counts()
print("The number of each phase:")
print(phase_counts)
print()

# count the number of drug name
drug_name_columns = [col for col in df.columns if 'Drug Name' in col]
drug_name_count_distribution = df[drug_name_columns].notna().sum(axis=1).value_counts().sort_index()

print("The number of each drug name:")
print(drug_name_count_distribution)

#Check whether drug name includes 'placebo' or not
df_str = df[drug_name_columns].astype(str)
contains_placebo = df_str.apply(lambda x: x.str.contains('Placebo|placebo', case=False, na=False)).any(axis=1)

num_contains_placebo = contains_placebo.sum()
num_not_contains_placebo = (~contains_placebo).sum()

print("The number of data entries with Placebo.:",num_contains_placebo)
print("The number of data entries without Placebo.:",num_not_contains_placebo)